<center> <h1>LoL-Model </h1> </center>
<center> <h2>Predicting wins in League of Legends</h2> </center>
<h3>Executive summary</h3>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam metus tellus, rhoncus a arcu id, molestie tempus nisl. Aliquam in turpis eget arcu volutpat pellentesque. Vestibulum aliquet ante ultricies, accumsan justo et, euismod diam. Suspendisse non nunc at odio lacinia imperdiet. Phasellus non lacus dapibus dui porttitor gravida ut id ante. Pellentesque in blandit leo. Vestibulum auctor mauris sit amet libero consequat, ut viverra mi imperdiet. Pellentesque nec ornare quam.

# Data preparation

## Libraries

In [47]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' %x)

## Data import

In [36]:
# Champions and ID
champions = pd.read_csv('data/champs.csv')

# Matches
matches = pd.read_csv('data/matches.csv')

# Player info
playersInfo = pd.read_csv('data/participants.csv')

# Stats
stat1 = pd.read_csv('data/stats1.csv')
stat2 = pd.read_csv('data/stats2.csv')
stats = stat1.append(stat2)

# Team - bans
bannedChampion = pd.read_csv('data/teambans.csv')

# Team - stats 
teamStats = pd.read_csv('data/teamstats.csv')

C:\Users\szmar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
## Create and clean the players table

# Merge player id with their stats
players = pd.merge(playersInfo, stats, on = ['id'], how = 'left', suffixes=('', '_y'))

# Add the champion name
players = pd.merge(players, champions, how = 'left', left_on = 'championid', right_on = 'id', suffixes=('', '_y'))

# Add the match technical info
players = pd.merge(players, matches, how = 'left', left_on = 'matchid', right_on = 'id', suffixes=('', '_y'))

# Add 1-5 players to team 1, 6-10 to team 2
players['team'] = players['player'].apply(lambda x: 1 if x <= 5 else 2)

In [46]:
players.head()

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,item2,item3,item4,item5,item6,trinket,kills,deaths,assists,largestkillingspree,largestmultikill,killingsprees,longesttimespentliving,doublekills,triplekills,quadrakills,pentakills,legendarykills,totdmgdealt,magicdmgdealt,physicaldmgdealt,truedmgdealt,largestcrit,totdmgtochamp,magicdmgtochamp,physdmgtochamp,truedmgtochamp,totheal,totunitshealed,dmgselfmit,dmgtoobj,dmgtoturrets,visionscore,timecc,totdmgtaken,magicdmgtaken,physdmgtaken,truedmgtaken,goldearned,goldspent,turretkills,inhibkills,totminionskilled,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood,name,id_y,id_y,gameid,platformid,queueid,seasonid,duration,creation,version,team
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,2003.0,3111.0,3053.0,1419.0,1042.0,3340.0,6.0,10.0,1.0,2.0,2.0,2.0,643.0,2.0,0.0,0.0,0.0,0.0,96980.0,25154.0,65433.0,6392.0,0.0,9101.0,3975.0,4237.0,888.0,15160.0,1.0,23998.0,1826.0,1170.0,14.0,0.0,41446.0,13270.0,24957.0,3218.0,10497.0,10275.0,0.0,0.0,42.0,69.0,42.0,27.0,610.0,13.0,0.0,0,10.0,0.0,0.0,Warwick,19,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,1
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,3111.0,3190.0,3107.0,0.0,0.0,3364.0,0.0,2.0,12.0,0.0,0.0,0.0,1116.0,0.0,0.0,0.0,0.0,0.0,25995.0,17633.0,6295.0,2066.0,0.0,8478.0,6684.0,977.0,816.0,11707.0,5.0,9402.0,1943.0,1852.0,30.0,0.0,17769.0,7945.0,7688.0,2136.0,9496.0,7975.0,1.0,0.0,17.0,1.0,1.0,0.0,211.0,14.0,1.0,0,17.0,3.0,0.0,Nami,267,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,1
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,3072.0,3006.0,3031.0,3046.0,1036.0,3340.0,7.0,8.0,5.0,5.0,2.0,1.0,584.0,2.0,0.0,0.0,0.0,0.0,171568.0,1725.0,169576.0,266.0,1042.0,14425.0,331.0,14070.0,24.0,2283.0,2.0,16612.0,5094.0,2128.0,26.0,0.0,25627.0,12538.0,11094.0,1993.0,13136.0,11775.0,0.0,0.0,205.0,3.0,1.0,2.0,182.0,14.0,1.0,0,13.0,5.0,0.0,Draven,119,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,1
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,3078.0,3156.0,1001.0,3053.0,0.0,3340.0,5.0,11.0,2.0,2.0,1.0,1.0,300.0,0.0,0.0,0.0,0.0,0.0,113721.0,989.0,109563.0,3168.0,455.0,15267.0,296.0,11802.0,3168.0,4252.0,1.0,27174.0,8263.0,8263.0,5.0,0.0,31705.0,10280.0,19506.0,1918.0,11006.0,10683.0,3.0,0.0,164.0,6.0,6.0,0.0,106.0,15.0,0.0,0,3.0,0.0,0.0,Fiora,114,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,1
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,1058.0,3198.0,3102.0,1052.0,1026.0,3340.0,2.0,8.0,2.0,0.0,1.0,0.0,504.0,0.0,0.0,0.0,0.0,0.0,185302.0,166671.0,16867.0,1763.0,0.0,18229.0,17925.0,28.0,275.0,1525.0,1.0,14616.0,3801.0,1724.0,15.0,0.0,20585.0,6850.0,11119.0,2615.0,11439.0,10485.0,1.0,0.0,235.0,4.0,3.0,1.0,159.0,15.0,0.0,0,10.0,0.0,0.0,Viktor,112,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,1


In [48]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834520 entries, 0 to 1834519
Data columns (total 74 columns):
id                        int64
matchid                   int64
player                    int64
championid                int64
ss1                       int64
ss2                       int64
role                      object
position                  object
win                       float64
item1                     float64
item2                     float64
item3                     float64
item4                     float64
item5                     float64
item6                     float64
trinket                   float64
kills                     float64
deaths                    float64
assists                   float64
largestkillingspree       float64
largestmultikill          float64
killingsprees             float64
longesttimespentliving    float64
doublekills               float64
triplekills               float64
quadrakills               float64
pentakills       